# SONATA for ringdown gravitational wave data analysis

In this notebook, we show an example of ringdown gravitational wave reconstruction with the SONATA method.
For more information on garvitational waves and its ringdown phase, please refer to the Section *V.B.* of the related (submitted) SSP article and references therein:

[Philippe __Flores__, Julien __Flamant__, Pierre-Olivier __Amblard__, Nicolas __Le Bihan__, _Damped ellipse decomposition for bivariate signals_. Submitted to SSP 2025.]

For more information or materials on this regard, please feel free to contact the authors by mail.

In [ ]:
## Package importations
import numpy as np
import pickle
import bispy as bsp

from sonata_visuals import *
from sonata_slra import *
from sonata_ellipse import *

#### Loading the ringdown data files

Along with the code for SONATA, this repository contains two synthetic ringdown data files.

* ./data/ringdown/no_spin
* ./data/ringdown/with_spin

The main difference between those two files is that the spins were set to 0 for the "no_spin" file. For each file, the different parameters are available in the "input.yaml" file.

In [ ]:
file_h_plus = open("data/ringdown/no_spin/hplus.pkl",'rb')
hplus = pickle.load(file_h_plus)
file_h_plus.close()

file_h_cross = open("data/ringdown/no_spin/hcross.pkl",'rb')
hcross = pickle.load(file_h_cross)
file_h_cross.close()

file_time = open("data/ringdown/no_spin/time.pkl",'rb')
t = pickle.load(file_time)
file_time.close()

#### Computation of the $\mathbb{H}$-embedding of the signal

The gravitational wave signal is a bivariate polarized signal defined by the two polarization states $h_+$ and $h_\times$. It is represented in our framework as a complex-valued signal $z[n] = h_+[n] - \mathbf{i} h_\times[n]$. SONATA studies bivariate signals via their $\mathbb{H}$-embedding $y[n]\in\mathbb{H}$ -- hence a quaternion-valued signal.

This section aims at computing the $\mathbb{H}$-embedding $y$ of the bivariate signal $z$. During this operation, the polar form of $y$ is computed:
$$ y[n] = a[n] e^{\mathbf{i}\theta[n]} e^{-\mathbf{k}\chi[n]} e^{\mathbf{j}\varphi[n]}, \quad 0\leq n < N.$$

In [ ]:
z = bsp.utils.sympSynth(hplus, -hcross)

z_as_H_embed = bsp.timefrequency.Hembedding(z)

a = z_as_H_embed.a
theta = z_as_H_embed.theta
chi = z_as_H_embed.chi
phi = z_as_H_embed.phi

y = bsp.utils.euler2quat(a,theta,chi,phi)

N = np.shape(y)[0]

#### Definition of starting time $t_0$ of the signal

The definition of $t_0$ lacks a unified answered from astro-physics experts. Indeed, this instant relates the exact moment from which the merger is finished and hence the moment from which the binary is no longer, giving place instead to the resulting black hole.

For this first experiment, we will consider the moment where $a[n]$ is maximum. For the "no-spin" file, this happens around  $t_0 = 0.0034s$ and $t_0 = 0.0013s$ for the "with-spin" file.

In [ ]:
xlim = [t[0],t[N-1]]
xlim = [0,0.03]

plt.figure(figsize = [10,3])

plt.subplot(1,4,1), plt.plot(t,a), plt.xlim(xlim), plt.title("$a[n]$")
plt.subplot(1,4,2)
plt.plot(t,theta), plt.xlim(xlim), plt.ylim([-np.pi/2,np.pi/2]), plt.title("$\\theta[n]$")
plt.subplot(1,4,3)
plt.plot(t,chi), plt.xlim(xlim), plt.ylim([-np.pi/4,np.pi/4]), plt.title("$\\chi[n]$")
plt.subplot(1,4,4)
plt.plot(t,phi), plt.xlim(xlim), plt.ylim([-np.pi,np.pi]), plt.title("$\\varphi[n]$")

plt.tight_layout()

In [ ]:
# t_0 = np.min(np.where(t>0.0034)) ## For the "no-spin" file
t_0 = np.min(np.where(t>0.0013)) ## For the "with-spin" file

t_end = 400

xlim = [0, t[t_end]]

plt.figure(figsize = [10,3])

plt.subplot(1,4,1)
plt.plot(t,a), plt.plot(t[t_0:t_end],a[t_0:t_end],'--'), plt.xlim(xlim), plt.title("$a[n]$")
plt.subplot(1,4,2)
plt.plot(t,theta), plt.plot(t[t_0:t_end],theta[t_0:t_end],'--'), plt.xlim(xlim), plt.ylim([-np.pi/2,np.pi/2]), plt.title("$\\theta[n]$")
plt.subplot(1,4,3)
plt.plot(t,chi), plt.plot(t[t_0:t_end],chi[t_0:t_end],'--'), plt.xlim(xlim), plt.ylim([-np.pi/4,np.pi/4]), plt.title("$\\chi[n]$")
plt.subplot(1,4,4)
plt.plot(t,phi), plt.plot(t[t_0:t_end],phi[t_0:t_end],'--'), plt.xlim(xlim), plt.ylim([-np.pi,np.pi]), plt.title("$\\varphi[n]$")

plt.tight_layout()
plt.show()

In [ ]:
## Irreversible, the signals and the time are defined between $t_0$ and $t_end$.
y = y[t_0:t_end]

t = t[t_0:t_end]

a = a[t_0:t_end]
theta = theta[t_0:t_end]
chi = chi[t_0:t_end]
phi = phi[t_0:t_end]


#### Performing SONATA on the synthetic ringdown data

For both $R = 1$ and $R = 5$, we reconstruct the synthetic ringdown signal with SONATA. The estimation are then plotted, both visualizing the whole reconstruction and each component.

In [ ]:
R = 1

M_hat_1, q_hat_1, flag_outer = sonata(y,R,number_inner_iterations=1)

plot_blind_estimated_ellipses(t,y,M_hat_1,q_hat_1)

In [ ]:
R = 5

M_hat_5, q_hat_5, flag_outer = sonata(y,R,number_inner_iterations=1)

plot_blind_estimated_ellipses(t,y,M_hat_5,q_hat_5)

#### Visualization of both reconstruction (see Figure 5 of the paper)

In [ ]:
y_1, y_2 = quaternion_to_complex(y)

y1 = rdot(M_hat_1,q_hat_1)
y1_1,y1_2 = quaternion_to_complex(y1)

y5 = rdot(M_hat_5,q_hat_5)
y5_1,y5_2 = quaternion_to_complex(y5)

plt.figure(figsize = [8,8])
gs = gridspec.GridSpec(12,4)
gs.update(hspace=0.5, wspace=1, bottom=0.1, left=0.1, top=0.95, right=0.95)

color_th = "#000000"
color_1 = "#ff7f0e"
color_5 = "#2ca02c"

ax1 = plt.subplot(gs[:7, :2])
ax1.plot(y_1.real,y_2.real, linewidth = 2,color = color_th)
ax1.plot(y1_1.real,y1_2.real,"--",linewidth = 3, color = color_1)

ax5 = plt.subplot(gs[:7,2:])
ax5.plot(y_1.real,y_2.real, linewidth = 2,color = color_th)
ax5.plot(y5_1.real,y5_2.real,"--",linewidth = 3, color = color_5)

ax_u = plt.subplot(gs[8:10,:])
ax_u.plot(t,y_1.real, linewidth = 3,color = color_th)
ax_u.plot(t,y1_1.real,":",linewidth = 3, color = color_1)
ax = ax_u.plot(t,y5_1.real,":",linewidth = 3,color = color_5)

ax_v = plt.subplot(gs[10:,:])
ax_v.plot(t,y_2.real, linewidth = 3,color = color_th)
ax_v.plot(t,y1_2.real,":",linewidth = 3, color = color_1)
ax_v.plot(t,y5_2.real,":",linewidth = 3,color = color_5)

ax1.set_xlabel("$h_+[n]$",fontsize = 15)
ax5.set_xlabel("$h_+[n]$",fontsize = 15)
ax_v.set_xlabel("$t[n]$",fontsize = 15)

ax1.set_ylabel("$h_\\times[n]$",fontsize = 15)
ax_u.set_ylabel("$h_+[n]$",fontsize = 15)
ax_v.set_ylabel("$h_\\times[n]$",fontsize = 15)

val_lim = np.max(np.abs(ax_u.get_ylim()+ax_v.get_ylim()))

ax_v.set_xlim([t[0],0.016])
ax_u.set_xlim([t[0],0.016])
ax1.set_xlim([-val_lim*0.75,val_lim*0.75])
ax5.set_xlim([-val_lim*0.75,val_lim*0.75])

ax1.set_ylim([-val_lim,val_lim*0.9])
ax5.set_ylim([-val_lim,val_lim*0.9])
ax_u.set_ylim([-val_lim,val_lim])
ax_v.set_ylim([-val_lim,val_lim])

ax_u.set_xticklabels([])

ax1.legend(["Synthetic ringdown","SONATA for $R = 1$"],loc = "upper center", fontsize = 15)
ax5.legend(["Synthetic ringdown","SONATA for $R = 5$"],loc = "upper center", fontsize = 15)

ax_u.grid()
ax_v.grid()
ax1.grid()
ax5.grid()

ax_v.legend(["Ringdown","$R = 1$","$R = 5$"],loc = "upper right", fontsize = 15,ncol = 3,bbox_to_anchor = (1,1.4))

ax1.set_title("$R = 1$",fontsize = 18)
ax5.set_title("$R = 5$",fontsize = 18)

plt.show()

In [ ]:
print("Reconstruction error for R = 1 : ", np.mean(abs(y-rdot(M_hat_1,q_hat_1))**2)/np.mean(abs(y)**2))
print("Reconstruction error for R = 5 : ", np.mean(abs(y-rdot(M_hat_5,q_hat_5))**2)/np.mean(abs(y)**2))